In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [110]:
def derivtransfer(FPs,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanh':
        derive=1/np.cosh(xe)**2
        derivi=1/np.cosh(xi)**2
        return np.array([derive,derivi])
def tanhpfunc(x):
    return np.tanh(x)+1.0
def checkXbase(FPs,J,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derxe=-xe+tanhpfunc(xe)*J[0,0]+tanhpfunc(xi)*J[0,1]
        derxi=-xi+tanhpfunc(xe)*J[1,0]+tanhpfunc(xi)*J[1,1]
        return (derxe,derxi)
    elif type=='tanh':
        derxe=-xe+np.tanh(xe)*J[0,0]+np.tanh(xi)*J[0,1]
        derxi=-xi+np.tanh(xe)*J[1,0]+np.tanh(xi)*J[1,1]
        return (derxe,derxi)
def FPfunc(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=np.tanh(x0+x1)+1
        resp1=np.tanh(x0-x1)+1
        return [
            x0-JE*resp0+JI*resp1,
            x1+a*resp0-b*resp1
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=(np.tanh(x0+x1))
        resp1=(np.tanh(x0-x1))
        return [
            x0-JE*resp0+JI*resp1,
            x1+a*resp0-b*resp1
        ]        

def FPXfunc(x,*data):
    JE,JI,a,b,tfunc=data
    # calculate connectivity matrix
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        respxE=tanhpfunc(x0)*Jt[0,0]+tanhpfunc(x1)*Jt[0,1]
        respxI=tanhpfunc(x0)*Jt[1,0]+tanhpfunc(x1)*Jt[1,1]
        return [
            x0-respxE,
            x1-respxI
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        respxE=np.tanh(x0)*Jt[0,0]+np.tanh(x1)*Jt[0,1]
        respxI=np.tanh(x0)*Jt[1,0]+np.tanh(x1)*Jt[1,1]
        return [
            x0-respxE,
            x1-respxI
        ]  

def FPfuncSVD(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=np.tanh(x0*m[0,0]+x1*m[0,1])+1
        resp1=np.tanh(x0*m[1,0]+x1*m[1,1])+1
        return [
            x0-n[0,0]*resp0-n[1,0]*resp1,
            x1-n[0,1]*resp0-n[1,1]*resp1
        ]
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=(np.tanh(x0*m[0,0]+x1*m[0,1])+0.0)
        resp1=(np.tanh(x0*m[1,0]+x1*m[1,1])+0.0)
        return [
            x0-n[0,0]*resp0-n[1,0]*resp1,
            x1-n[0,1]*resp0-n[1,1]*resp1
        ]

In [3]:
# bseries= np.logspace(-2,  2.0,num =20,base=2)
JE,JI,a,b=1.6,0.8,1.0,0.20
jeseries = np.linspace(2.40,  4.20,num =100)  
nlen=len(jeseries)
M=np.array([[1,1],[1,-1]])
xFPseries = np.zeros((nlen,2,2))
svdvalues = np.zeros((nlen,2))
svdvec = np.zeros((nlen,2,4))
kappaMN,kappamnSVD=np.zeros((nlen,2)),np.zeros((nlen,2))
epsErr=1e-5
for idxje,JE in enumerate(jeseries):
    # calculate SVD
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
    if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
        xFPseries[idxje,idxb,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue   
    lvec,sv,rvech=la.svd(Jt)
    svdvalues[idxje,:]=sv
    m,n=lvec,rvech.T
            
    m[:,0]*=sv[0]
    m[:,1]*=sv[1]
    if n[0,0]<0:
        n[:,0]*=(-1)   
        m[:,0]*=(-1)
    if n[0,1]<0:
        n[:,1]*=(-1)   
        m[:,1]*=(-1)
    svdvec[idxje,:,:2]=m
    svdvec[idxje,:,2:]=n

    N=np.array([[JE,-a],[-JI,b]])
    data=(JE,JI,a,b,'tanhp')
    x0=[10.0,10.0]
    results = fsolve(FPfunc,x0,data)
    kappaMN[idxje,:]=results
    xFP= M@np.reshape(results,(2,1))
    # check the solution
    errE,errI=checkXbase(xFP,Jt,type='tanhp')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,0,:]=np.nan
        kappaMN[idxje,:]=np.nan
        continue
    xFPseries[idxje,0,:]=xFP[:,0]
    
    resultSVD= fsolve(FPfuncSVD,x0,data)
    kappamnSVD[idxje,:]=resultSVD
    xFPSVD= m@np.reshape(resultSVD,(2,1))
    # check the solution
    errE,errI=checkXbase(xFPSVD,Jt,type='tanhp')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,1,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue
    xFPseries[idxje,1,:]=xFPSVD[:,0]
    # check eig
    Xderive=derivtransfer(xFPSVD,type='tanh')
    Jcheck=Jt.copy()
    Jcheck[:,0]*=Xderive[0]
    Jcheck[:,1]*=Xderive[1]
    eigv,eigvec=la.eig(Jcheck)
    if eigv[0]>1.0:
        xFPseries[idxje,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue
    # print('Fixed points A:',xFP,' SVD(B):',xFPSVD)
    # overlap under MNT
    diagdphi=np.zeros((2,2))
    for i in range(2):
        diagdphi[i,i]=1/np.cosh(xFPSVD[i])**2
    stabilityMN=N.T@diagdphi@M#
    stabilitymn=n.T@diagdphi@m#
    eigvMN,eigvecMN=la.eig(stabilityMN)
    eigvmn,eigvecmn=la.eig(stabilitymn)
        # print('eigenvalues of overlap MN and mn:',eigvMN,' and ',eigvmn)


In [4]:
fig=plt.figure()

ax1=fig.add_subplot(2,2,1)
ax1.plot(jeseries,xFPseries[:,0,0],'r')
ax1.plot(jeseries,xFPseries[:,0,1],'b')

ax2=fig.add_subplot(2,2,2)
ax2.plot(jeseries,xFPseries[:,1,0],'r')
ax2.plot(jeseries,xFPseries[:,1,1],'b')

ax3=fig.add_subplot(2,2,3)
ax3.plot(jeseries,kappaMN[:,0],'r')
ax3.plot(jeseries,kappaMN[:,1],'b')
ax4=fig.add_subplot(2,2,4)
ax4.plot(jeseries,kappamnSVD[:,0],'r')
ax4.plot(jeseries,kappamnSVD[:,1],'b')

In [5]:


ax1.plot(jeseries,xFPseries[:,0,0],'r--')
ax1.plot(jeseries,xFPseries[:,0,1],'b--')
ax1.set_title('reconstructed from (M,N)')

ax2.plot(jeseries,xFPseries[:,1,0],'r--')
ax2.plot(jeseries,xFPseries[:,1,1],'b--')
ax2.set_title('reconstructed from (m,n)')
ax1.set_ylabel(r'$x$')
ax2.set_ylabel(r'$x$')

ax3.plot(jeseries,kappaMN[:,0],'r--')
ax3.plot(jeseries,kappaMN[:,1],'b--')
ax3.set_ylabel(r'$\kappa^{(M)}$')
ax3.set_xlabel(r'$J_E$')

ax4.plot(jeseries,kappamnSVD[:,0],'r--')
ax4.plot(jeseries,kappamnSVD[:,1],'b--')
ax4.set_ylabel(r'$\kappa^{(m)}$')
ax4.set_xlabel(r'$J_E$')

Text(0.5, 0, '$J_E$')

In [103]:
# Noncline analysis --- intersection of two lines
JE,JI,a,b=3.0,0.6,1.2,-0.2
Jt = np.zeros((2,2))
Jt[:,0],Jt[:,1]=JE,-JI
Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
# Use neuronal responses X
xE,xI=np.linspace(-8.0,8.0,320),np.linspace(-8.0,8.0,320)
[XE,XI]=np.meshgrid(xE,xI)
# Surface of rhs in ODEs
rhsE,rhsI=Jt[0,0]*tanhpfunc(XE)+Jt[0,1]*tanhpfunc(XI),Jt[1,0]*tanhpfunc(XE)+Jt[1,1]*tanhpfunc(XI)
# rhsE,rhsI=Jt[0,0]*np.tanh(XE)+Jt[0,1]*np.tanh(XI),Jt[1,0]*np.tanh(XE)+Jt[1,1]*np.tanh(XI)
epsdis = 2*1e-4#((xE[1]-xE[0])/5.0)**2
# fig=plt.figure()
# ax1=fig.add_subplot(111)#,projection='3d') 
# # ax1.plot_surface(XE, XI, rhsE, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
# # ax1.plot_surface(XE, XI, XI, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
# ax1.contour(XE, XI, XI, zdir = 'y', offset = 0, cmap = plt.get_cmap('rainbow'))
# ax1.contour(XE, XI, rhsE, zdir = 'y', offset = 0, cmap = plt.get_cmap('rainbow'))
# finding the intersection
distrhsE=(rhsE-XE)**2
intersectE=np.where(distrhsE<epsdis)
# fig=plt.figure()
# ax2=fig.add_subplot(111)#,projection='3d') 
# # ax2.plot_surface(XE, XI, rhsI, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
# # ax2.plot_surface(XE, XI, XE, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
# ax2.contour(XE, XI, XE, zdir = 'y', offset = 0, cmap = plt.get_cmap('rainbow'))
# ax2.contour(XE, XI, rhsI, zdir = 'y', offset = 0, cmap = plt.get_cmap('rainbow'))

distrhsI=(rhsI-XI)**2
intersectI=np.where(distrhsI<epsdis)

fig=plt.figure()
ax1=plt.axes(projection='3d')
# ax1.plot3D(XE[intersectE],XI[intersectE],rhsE[intersectE],'gray')
# ax1.plot3D(XE[intersectE],XI[intersectE],XI[intersectE],'gray')

# ax1.plot_surface(XE, XI, rhsE, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
# ax1.plot_surface(XE, XI, XE, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
ax1.contour(XE, XI, XE, zdir = 'y', offset = 8.0, cmap = plt.get_cmap('rainbow'))
ax1.contour(XE, XI, rhsE, zdir = 'y', offset = 8.0, cmap = plt.get_cmap('rainbow'))
# ax1.plot3D(XE[intersectI],XI[intersectI],rhsI[intersectI],'blue')
# ax1.plot3D(XE[intersectI],XI[intersectI],XI[intersectI],'blue')
fig=plt.figure()
ax1=plt.axes()
ax1.plot(xE[intersectI[0]],xI[intersectI[1]],'b')
ax1.plot(xE[intersectE[0]],xI[intersectE[1]],'r')

In [42]:
fig=plt.figure()
ax1=fig.add_subplot(111,projection='3d') 
ax1.plot_surface(XE, XI, rhsE, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))
# ax1.contour(XE, XI, rhsE, zdir = 'z', offset = -2, cmap = plt.get_cmap('rainbow'))
ax1.plot_surface(XE, XI, XI, rstride = 1, cstride = 1, cmap = plt.get_cmap('rainbow'))

In [109]:
# bseries= np.logspace(-2,  2.0,num =20,base=2)
JE,JI,a,b=3.0,0.6,1.2,-0.2
# calculate SVD
Jt = np.zeros((2,2))
Jt[:,0],Jt[:,1]=JE,-JI
Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
lvec,sv,rvech=la.svd(Jt)
m,n=lvec,rvech.T
        
m[:,0]*=sv[0]
m[:,1]*=sv[1]
if n[0,0]<0:
    n[:,0]*=(-1)   
    m[:,0]*=(-1)
if n[0,1]<0:
    n[:,1]*=(-1)   
    m[:,1]*=(-1)

N=np.array([[JE,-a],[-JI,b]])
data=(JE,JI,a,b,'tanhp')
kappa0=[0.0,0.0]
results = fsolve(FPfunc,kappa0,data)
xFP= M@np.reshape(results,(2,1))
x0=[0.0,0.0]
resultx = fsolve(FPXfunc,x0,data)

resultSVD= fsolve(FPfuncSVD,x0,data)
xFPSVD= m@np.reshape(resultSVD,(2,1))
print('ss:',xFPSVD,'; directly X:',resultx)



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices